# Preprocessing

Like other data types, text data never comes clean. Moreover, most of our downstream methods only accept data structured in a particular way. Because of this, before we do any computational text analysis techniques, we will always need to perform some level of preprocessing on our text data. 

Different applications require different approaches to preprocessing. There is no "one size fits all" approach, and even if you use a program or module with built-in preprocessing steps, you want to know what it's doing (and not doing) so you can justify and/or change it. In some cases, it's better to do minimal preprocessing, to retain the valuable meanings stored in sentence structure and the like. In other cases, we want only a "bag of words", so more preprocessing is better. There are as many choices in preprocessing as there are in analysis (lots and lots!), so it's essential to understand the building blocks. 

For instance, to do corpus linguistics (e.g., syntactic parsing) or neural network models (e.g., word embeddings), we usually want to keep sentence structure and different word forms, so we would just tokenize and lower-case words, segment sentences, and normalize text (remove URLs, etc.). In contrast, for simpler techniques like finding the most distinctive or frequent words--as well as for topic models--we would typically do all that plus remove stopwords and punctuation and stem or lemmatize words. 

To help you understand these choice points so you can assemble your own domain-appropriate preprocessing recipes, in this notebook we will cover the basic ingredients.

## Learning Goals
* Get an introduction to the Python package NLTK, what functionality it offers, and learn some important NLP terms
* Learn about the major choice points in preprocessing--and why you might or might not do them
* Learn how to do a variety of forms of counting using NLTK, and think about why these might help researchers analyze text
* Practice the basic ingredients in text preprocessing

## Outline
(TO DO: Add hyperlinks for each section)

* [Reading in .txt and .csv files](#read)
* [Tokenizing words](#tokenize)
* [Sentence segmentation](#sentence)
* [Removing punctuation](#punctuation)
* [Stripping whitespace](#whitespace)
    * Start and end of string
    * Between words
* [Text normalization](#normalize)
    * Case folding
    * Removing URLs, digits, and hashtags
* [Counting word frequencies](#freqs)
* [Removing stop words](#stopwords)
* [Stemming/Lemmatizing](#stem)
* [Parts-of-speech (POS) tagging](#POS)
* [Document-Term Matrix (DTM)](#DTM)

## Background

We will do some review, but this notebook assumes you have basic familiarity with Python. If you need a beginner's introduction to coding in Python, please walk through the intro to Python notebook at `solutions/intro-to-python.ipynb` and/or [this one](https://github.com/lknelson/text-analysis-course/blob/master/scripts/01.25.02_PythonBasics.ipynb) *before* the workshop. 

We will also use some regular expressions, which are character sequences defining a search pattern. Usually this pattern is then used by string searching algorithms for "find" or "find and replace" operations. Don't worry if you haven't seen these before--we will keep it simple. If you want to get more out of this session, go through [this notebook on regular expressions](https://github.com/lknelson/text-analysis-course/blob/master/scripts/03.20.01_RegularExpressions.ipynb).

## Vocabulary

* *stop words*: 
    * The most common words in a language.
* *tokenization*:
    * Given a character sequence and a defined document unit, tokenization is the task of chopping it up into pieces, called tokens , perhaps at the same time throwing away certain characters, such as punctuation. 
* *token*:
    *  A token is an instance of a sequence of characters in some particular document that are grouped together as a useful semantic unit for processing.
* *type*:
    * A type is the class of all tokens containing the same character sequence.

## Further Resources

This is a great starting point, but there is a lot more to learn about preprocessing. To acquaint yourself with a few more tools, check out `solutions/preprocessing_extra.ipynb`, the other resources listed in [the workshop repo](https://github.com/jhaber-zz/nlp-python-2020), and the [useful intro scripts prepared by my colleague Laura Nelson](https://github.com/lknelson/text-analysis-course). To gain more practice and tools, come back for tomorrow's session!

[Denny, Matthew J., and Arthur Spirling. 2018. “Text Preprocessing for Unsupervised Learning: Why It Matters, When It Misleads, and What to Do about It.” Political Analysis 26(2):168–189.
](https://www.cambridge.org/core/journals/political-analysis/article/text-preprocessing-for-unsupervised-learning-why-it-matters-when-it-misleads-and-what-to-do-about-it/AA7D4DE0AA6AB208502515AE3EC6989E)
<br>Documents importance of preprocessing choices to modeling results.

**__________________________________**


## Reading in files<a id='read'></a>

The first step is to read in the files containing the text data. The most common file types for text data are: `.txt`, `.csv`, `.json`, `.html` and `.xml`.

### Reading in `.txt` files

Python has built-in support for reading in `.txt` files.

In [ ]:
import os
DATA_DIR = 'data'
fname = 'pride-and-prejudice.txt'
fname = os.path.join(DATA_DIR, fname)
with open(fname, encoding='utf-8') as f:
    raw = f.read()

#### Review of Python string methods

- What type of object is `raw`?
- How many characters are in `raw`?
- Get the first 1000 characters of `raw`.
- Join together the first 200 and the last 200 characters of `raw`.

In [ ]:
# your code here

### Reading in `.csv`

Python has a built-in module called `csv` for reading in csv files.

In [ ]:
import csv
fname = 'trump-tweets.csv'
fname = os.path.join(DATA_DIR, fname)
tweets = []

# standard approach to opening files (no special encoding situations): # with open(fname) as f:
import codecs
with codecs.open(fname, "r", encoding='utf-8', errors='ignore') as f: # for special encoding issues  
    reader = csv.reader(f)
    tweets = list(reader)

#### Review of Python list methods

- What data type is `tweets`?
- How many entries are in `tweets`?
- Which entry is the header row?
- Get the first 10 entries.
- Join together the 5th and 10th elements of `tweets`.

In [ ]:
# your code here

### Reading in `.csv` with `pandas`

`pandas` is a third-party library that makes working with tabular data much easier. This is the recommended way to read in a `.csv` file.

In [ ]:
import os
import pandas as pd
fname = 'trump-tweets.csv'
fname = os.path.join(DATA_DIR, fname)
tweets = pd.read_csv(fname) 

#### Review of `pandas`

- What data type is `tweets`?
- How many tweets are there?
- What happened to the header row?
- Get the first row of `tweets`.
- Get the first 5 entries in the `Tweet_Text` column.

In [ ]:
# your code here

### Reading in multiple files

Often, our text data is split across multiple files in a folder. We want to read them all into a single variable. <br>`glob` is a handy package for this: it lists all files matching a pattern. We can use this to get all files in a folder. 

In [ ]:
import glob
fnames = os.path.join(DATA_DIR, 'austen', '*.txt')
fnames = glob.glob(fnames)
austen = ''
for fname in fnames:
    with codecs.open(fname, "r", encoding='utf-8-sig', errors='ignore') as f:
        text = f.read()
        austen += text

#### Review of working with files

- What does `os.path.join()` do in this case?
- What type is `fnames` after it is first assigned a value?
- What type is `fnames` after it is assigned a second value?
- How many files are in `fnames`?
- What type is `austen`?

In [ ]:
# your code here

### Challenge 

Read in all the `.csv` files in the folder `amazon`. Extract out only the `text` column from THE FIRST TWO files and store them all in a list called `reviews`. 

**Hint 1:** Not all of these files heave a header row to indicate column names. But for your reference, the columns are in this order: <br>
```Id, ProductId, UserId, ProfileName, HelpfulnessNumerator, HelpfulnessDenominator, Score, Time, Summary, Text```

**Hint 2:** You can deal with `.csv` files without header rows by calling the argument `header=None` when loading into a pandas DataFrame. This lets pandas know not to mistake the first row of data for column names. 

In [ ]:
# your solution here

## Tokenization<a id='tokenize'></a>

Once we've read in the data, our next step is often to split it into words. This step is referred to as "tokenization", because each word occurrence is referred to as a "token". Each distinct word used is called a word "type". So the word type "the" may correspond to multiple tokens of "the" in a text. For example, the sentence "the dog ate the cat" has five tokens ("the", "dog", "ate", "the", "cat") but only *four* types ("the", "dog", "ate", "cat").

There are many, many approaches to tokenization. The simplest approach to splitting words is by whitespace--the spacing between words. Let's see how to do that and compare it to a few other basic and accessible methods.

### Tokenizing by whitespace

In [ ]:
import os
fname = 'example1.txt'
fname = os.path.join(DATA_DIR, fname)
with open(fname) as f:
    text = f.read()

In [ ]:
text

In [ ]:
text.split()[:20] # tokenize by whitespace

- What problems do you notice with tokenizing by whitespace? If we're just looking for a "bag of words", what other steps should we take?
- What type is `text`?

### Tokenizing with regular expressions

In [ ]:
import re
word_pattern = r'\w+' # \w means "one or more words": https://www.debuggex.com/cheatsheet/regex/python
tokens = re.findall(word_pattern, text)
tokens[:20]

- What type is `tokens`?
- What type is each element of `tokens`?
- What does tokenizing with regular expressions do that tokenizing with whitespace does not?

### Tokenizing with `nltk`

[Just a bunch of regular expressions under the hood](https://github.com/nltk/nltk/blob/develop/nltk/tokenize/treebank.py)

In [ ]:
from nltk.tokenize import word_tokenize
import nltk; nltk.download('punkt')
tokens = word_tokenize(text)
tokens[:20]

- How does tokenization with NLTK compare to the previous approaches?

### Challenge

A while ago you read in a bunch of Jane Austen books into a variable called `austen`. Tokenize that using a method of your choice. Find all the unique words types (you might want the `set` function). Sort the resulting set object to create a vocabulary (you might want to use the `sorted` function). 

**Extra credit:** Use what you know about types and tokens to measure lexical uniqueness--that is, how often Jane Austen reuses words. <br>*Hint:* Start by counting the number of types and the number of tokens in `austen`.

In [ ]:
# your solution here

## Sentence segmentation<a id='sentence'></a>

Sentence segmentation involves identifying the boundaries of sentences.

### Sentence segmentation by splitting on periods

In [ ]:
text.split('.')

- What does this simple approach do well? What does it miss?

### Sentence segmentation with regular expressions

Regular expressions allow us to split strings based on a number of characters.

In [ ]:
sent_boundary_pattern = r'[.?!]' # This pattern identifies at least one punctuation mark concluding a sentence
re.split(sent_boundary_pattern, text)

### Sentence segmentation with `nltk`

In [ ]:
from nltk.tokenize import sent_tokenize
sent_tokenize(text)

### Challenge - SOLUTION

The file `example2.txt` has more punctuation problems. Read it in (using same pattern as above) and see what the problems are. Try the different approaches above and see what works best. If you're already familiar with regular expressions, modify that code to work for as many cases as you can. What does this suggest about tokenizing text that includes punctuation?
<br> (For help with regular expressions, see this [cheat sheet](https://www.debuggex.com/cheatsheet/regex/python))

In [ ]:
# your solution here

## Removing punctuation<a id='punctuation'></a>

In some situations, it's best to keep only the alphanumeric characters (i.e. the letters and numbers) and ditch the punctuation. Here's how we can do that.

In [ ]:
from string import punctuation
punctuation

- What type is `punctuation`?

In [ ]:
no_punct = ''.join([char for char in text if char not in punctuation])
no_punct

## Strip whitespace<a id='whitespace'></a>

This is an extremely common step. It's simple to perform and nicely pre-packaged in Python. It's particularly common for user-generated text (think survey forms).

In [ ]:
string = ' Hello! '
string.strip()

In [ ]:
fname = 'example3.txt'
fname = os.path.join(DATA_DIR, fname)
with open(fname) as f:
    text = f.read()
print(text) # This shows the whitespaces

### Start and end of string

In [ ]:
stripped_text = text.strip() # This just strips whitespace from front and end--what about in between words?
print(stripped_text)

### Between words

In [ ]:
# To strip whitespaces both at extremes and inside a text, regular expressions are a great tool:
whitespace_pattern = r'\s+'
clean_text = re.sub(whitespace_pattern, ' ', text)
clean_text.strip() # Strip whitespaces at start & end (again)

## Text normalization<a id='normalize'></a>

Text normalization means making our text fit some standard patterns. Lots of steps come under this wide umbrella, but the most common are:

- case folding
- removing URLs, digits, hashtags
- removing infequent words (not done here)

### Case folding

Case folding means dealing with upper and lower cases characters. This is usually done by making all characters lower-cased.

In [ ]:
fname = 'example4.txt'
fname = os.path.join(DATA_DIR, fname)
with open(fname) as f:
    text = f.read()
text

In [ ]:
text.lower()

### Challenge
The `lower` method we used above is a string method, that is, it works on strings. But say you've already tokenized a test, and you want to lowercase every word in the resulting list? Take the list of tokens below and make each one lower case.

In [ ]:
tokens = word_tokenize(text)
tokens[:20]

In [ ]:
# your solution here

### Removing URLs, digits, and hashtags

We rarely care about the exact URL used in a tweet, or the exact number of tweets. We could remove them completely, but it's often informative to know that there is a URL or a digit in the text using a placeholder (to avoid clouding our measures with the actual content). So we want to replace individual URLs and digits with a symbol that preserves the fact that a URL was there. It's standard to just use the strings "URL" and "DIGIT".

How do we do this? Once again, regular expressions save the day.

In [ ]:
# First, get just the text for the tweets we read in earlier:
tweet_text = tweets["Tweet_Text"] # 

In [ ]:
url_pattern = r'https?:\/\/.*[\r\n]*'
single_tweet = tweet_text.iloc[0]
single_tweet

In [ ]:
URL_SIGN = ' URL '
re.sub(url_pattern, URL_SIGN, single_tweet)

Above we replaced the URL in a single tweet. Now we will replace all the URLs for each tweet in `tweet_text`.

In [ ]:
url_pattern = r'https?:\/\/.*[\r\n]*'
URL_SIGN = ' URL '
list_of_url_less_tweets = []

# Use a for loop
for tweet in tweet_text:
    url_less_tweet = re.sub(url_pattern, URL_SIGN, tweet)
    list_of_url_less_tweets.append(url_less_tweet)
list_of_url_less_tweets

In [ ]:
## Alternative using list comprehension
list_of_url_less_tweets = [re.sub(url_pattern, URL_SIGN, tweet) for tweet in tweet_text]
list_of_url_less_tweets

Now let's remove hashtags and digits.

In [ ]:
hashtag_pattern = r'(?:^|\s)[＃#]{1}(\w+)'
HASHTAG_SIGN = ' HASHTAG '
digit_pattern = '\d+'
DIGIT_SIGN = ' DIGIT '

In [ ]:
no_hashtags = [re.sub(hashtag_pattern, HASHTAG_SIGN, tweet) for tweet in tweet_text]
no_hashtags

In [ ]:
no_digit = [re.sub(digit_pattern, DIGIT_SIGN, tweet) for tweet in tweet_text]
no_digit

## Counting word frequencies<a id='freqs'></a>

We can count the frequency of each word type with the built-in `Counter` in Python. This basically just takes the set of word types (we calculated this above as `vocabulary`) and makes a special Python dictionary with each value being the number of times it appears in the list. We can ask that dictionary for the most common words, or for the frequency of individual word types. 

First, clean and normalize the text:

In [ ]:
all_tweets = ' '.join(tweets)
clean = re.sub(url_pattern, URL_SIGN, all_tweets)
clean = re.sub(hashtag_pattern, HASHTAG_SIGN, clean)
clean = re.sub(digit_pattern, DIGIT_SIGN, clean)
tokens = word_tokenize(clean)
tokens = [token for token in tokens if token not in punctuation]
tokens[:20]

In [ ]:
from collections import Counter
freq = Counter(tokens)
freq.most_common(10)

Another common way to count frequent words is with a built-in NLTK module:

In [ ]:
#apply the nltk function FreqDist to count the number of times each token occurs.
from nltk import FreqDist
word_frequency = FreqDist(tokens)

#this creates an NLTK object
print(word_frequency)

In [ ]:
print(word_frequency.most_common(10))

### Challenge 

I've read in some Amazon reviews from earlier into a list called `reviews`. Each element of the list is a string, representing the text of a single review. Try to:
- Tokenize each review
- Strip all whitespace
- Make all characters lower case
- Replace any URLs and digits

Then find the most common 50 words.

In [ ]:
fnames = os.path.join(DATA_DIR, 'amazon', '*.csv')
fnames = glob.glob(fnames)
reviews = []
column_names = ['id', 'product_id', 'user_id', 'profile_name', 'helpfulness_num', 'helpfulness_denom',
               'score', 'time', 'summary', 'text']
for fname in fnames[:2]:
    df = pd.read_csv(fname, names=column_names)
    text = list(df['text'])
    reviews.extend(text)

In [ ]:
# your solution here

## Removing stop words<a id='stopwords'></a>

You might have noticed that the most common words above aren't terribly exciting. They're words like "am", "i", "the" and "a": stop words. These are rarely useful to us in computational text analysis, so it's very common to remove them completely.

- What other stop words do you think there are?

In [ ]:
import nltk; nltk.download('stopwords')
from nltk.corpus import stopwords
stop = stopwords.words('english')
stop

### Challenge 

Use the list `stop` of English stopwords to remove stopwords from our tokenized review above.

In [ ]:
# your solution here

## Stemming/lemmatization<a id='stem'></a>

Stemming and lemmatization both refer to remove morphological affixes on words. For example, if we stem the word "grows", we get "grow". If we stem the word "running", we get "run". We do this because often we care more about the core content of the word (i.e. that it has something to do with growth or running, rather than the fact that it's a third person present tense verb, or progressive participle).

NLTK provides many algorithms for stemming. For English, a great baseline is the [Porter](https://github.com/nltk/nltk/blob/develop/nltk/stem/porter.py) algorithm, which is in spirit isn't that far from a bunch of regular expressions.

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

In [ ]:
stemmer.stem('grows')

In [ ]:
stemmer.stem('running')

In [ ]:
stemmer.stem('leaves')

In [ ]:
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import nltk; nltk.download('wordnet') # Download resource for working with WordNet via NLTK
snowballer_stemmer = SnowballStemmer('english')
lemmatizer = WordNetLemmatizer()

In [ ]:
print(snowballer_stemmer.stem('running'))
print(snowballer_stemmer.stem('leaves'))

In [ ]:
print(lemmatizer.lemmatize('leaves'))

### Challenge 

Use the Porter stemmer to stem each word in the tweet dataset after having removed stop words.

In [ ]:
# your solution here

## POS tagging<a id='POS'></a>

POS tagging means assigning each token a part-of-speech (e.g. noun, verb, adjective, etc.). Again, there are many different [alternatives](https://github.com/nltk/nltk/tree/develop/nltk/tag), but NLTK keeps its recommended POS tagger available through the function `pos_tag`. The tagger expects a list of tokens as input.When doing POS tagging, it is advisable **not** to remove stop words beforehand (although you are free to do it afterwards).

In [ ]:
from nltk import pos_tag
single_review = reviews[3]
single_review

In [ ]:
tokens = word_tokenize(single_review)
import nltk; nltk.download('averaged_perceptron_tagger')
tagged_review = pos_tag(tokens)
tagged_review

### Challenge 

Below I've read in the text of Austen's _Pride and Prejudice_ into a variable called `pride`. Preprocess using the following steps:

- Strip whitespace
- Replace all numbers with '0'
- Tokenize
- Tag each token with a POS tag

Make sure you know:
- What type is the result?
- What type is each element of the result?
- What type are the elements of the elements of the result?

In [ ]:
fname = 'pride-and-prejudice.txt'
fname = os.path.join(DATA_DIR, fname)
with open(fname, encoding='utf-8') as f:
    raw = f.read()
pride = raw[679:684814]
pride

In [ ]:
# your solution here

## Document-Term Matrix<a id='DTM'></a>



## Things we didn't cover
(see `solutions/preprocessing_extra.ipynb` and [this repo](https://github.com/geoffbacon/nlp-with-nltk-spacy/blob/master/03-NLTK.ipynb) for more on these)

- Reading in JSON, HTML, and XML files 
- Removing infrequent words
- Named entity recognition
- Scikit-learn preprocessing pipelines
- Syntactic parsing
- Information extraction
- Removing markup from HTML
- Extracting numerical features
- Term Frequency Inverse Document Frequency (TF-IDF)
- SpaCy